# 1.加载配置文件

In [1]:
from sentence_transformers import SentenceTransformer
import os
from openai import AzureOpenAI
import json
from aip import AipSpeech

# OPENAI-API配置
AZURE_OPENAI_API_KEY = '4213f3298d82495a8fdaf5e838402493'
AZURE_OPENAI_ENDPOINT = 'https://yidu-resource-4.openai.azure.com/'
os.environ['AZURE_OPENAI_API_KEY'] = AZURE_OPENAI_API_KEY
os.environ['AZURE_OPENAI_ENDPOINT'] = AZURE_OPENAI_ENDPOINT
os.environ['OPENAI_API_VERSION'] = '2024-05-01-preview'

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

def load_model(model_path):
    return SentenceTransformer(model_path)

def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def encode_questions(embedding_model, questions):
    return embedding_model.encode(questions, normalize_embeddings=True)

# 加载本地embedding模型
embedding_model_path = 'C:/Users/AizZHOU/Desktop/Github-Audio2Text2SQL/model/Dmeta-embedding-zh'
embedding_model = load_model(embedding_model_path)

all_data_path = 'C:/Users/AizZHOU/Desktop/Github-Audio2Text2SQL/dataset/all_data_process_unique.json'
all_data = load_json(all_data_path)

questions = [item['question'] for item in all_data]
queries = [item['query'] for item in all_data]
mask_queries = [item['mask_query'] for item in all_data]

question_embeddings = encode_questions(embedding_model, questions)

# 加载百度asr模型
BAIDU_APP_ID = '97177283'
BAIDU_API_KEY = '1WwRhOk8CQ35IxEOhya7PStk'
BAIDU_SECRET_KEY = 'qFHoynE4YAnhPVDI1NKTLXVhmjJHdd27'
aip_speech = AipSpeech(BAIDU_APP_ID, BAIDU_API_KEY, BAIDU_SECRET_KEY)

d:\python\envs\asr\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# 2.输出sql

In [4]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import re
import time
import speech_recognition as sr
import logging
from pynput import mouse

# 嵌入模型
def encode_questions(embedding_model, questions):
    return embedding_model.encode(questions, normalize_embeddings=True)

def compute_distances(question_embedding, question_embeddings):
    return euclidean_distances(question_embedding, question_embeddings).squeeze()

def extract_sql(query):
    query = query.replace("### SQL:", "").replace("###", "").replace("#", "")
    sql_patterns = [r"```SQL(.*?)```", r"```SQL(.*?)", r"```sql(.*?)```", r"```sql(.*?)"]
    for pattern in sql_patterns:
        matches = re.findall(pattern, query.replace('\n', ' '))
        if matches:
            return matches[0].strip()
    return query.replace('\n', '').replace("`", "").strip()

def generate_prompt(examples, question):
    return f"""
### 以下是基于类似问题提供的一些问题和相应的SQL查询的示例对：
{examples}
### 仅通过SQLite SQL查询回答问题，不需要解释。您必须在确保正确性的同时最小化SQL执行时间。 
### 给定以下数据库架构:
#
# CREATE TABLE t_di_bind_device (id INT PRIMARY KEY AUTO_INCREMENT COMMENT '唯一id',user_id VARCHAR(20) NOT NULL COMMENT '用户手机号，一个用户可绑定多个设备',mac_id CHAR(16) NOT NULL UNIQUE COMMENT '设备id，设备id长度一般为16位，每个设备都有唯一id',mac_type VARCHAR(50) COMMENT '设备型号',channel_id VARCHAR(50) COMMENT '设备渠道',bind TINYINT(1) NOT NULL COMMENT '设备绑定状态，设备绑定：1；设备解绑：0。一个设备仅有一个主绑定用户',created TIMESTAMP DEFAULT CURRENT_TIMESTAMP COMMENT '记录创建时间',modified TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '记录修改时间');
# CREATE TABLE t_device_attributes (mac_id CHAR(16) NOT NULL UNIQUE COMMENT '设备id，设备id长度一般为16位，每个设备都有唯一id',mac_type VARCHAR(50) COMMENT '设备型号',supplier_id VARCHAR(50) COMMENT '厂商id',supplier_name VARCHAR(100) COMMENT '厂商名称',channel_name VARCHAR(100) COMMENT '渠道名称',channel_id VARCHAR(50) COMMENT '渠道id',province_code VARCHAR(10) COMMENT '省码',province_name VARCHAR(100),PRIMARY KEY (mac_id));
# CREATE TABLE t_user_packages (order_id CHAR(32) NOT NULL PRIMARY KEY COMMENT '订单id，每个订单都有唯一id',mac_id CHAR(16) NOT NULL COMMENT '设备id，设备id长度一般为16位，每个设备都有唯一id',order_user VARCHAR(20) NOT NULL COMMENT '订购手机号，一个设备可绑定一个手机号、开通一种套餐',province_code VARCHAR(10) COMMENT '省码',city_code VARCHAR(10) COMMENT '市码',created TIMESTAMP DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',package_id CHAR(32) NOT NULL COMMENT '套餐id，每一类套餐具有唯一id，长度一般为32位',storage_type TINYINT(1) NOT NULL COMMENT '套餐类型，全天套餐：1，事件套餐：2',state TINYINT(1) NOT NULL COMMENT '套餐状态，当前套餐处于生效状态：1，当前套餐处于失效状态：0');
# CREATE TABLE t_device_online_status (id INT PRIMARY KEY AUTO_INCREMENT COMMENT '序号',mac_id CHAR(16) NOT NULL COMMENT '设备id',login TINYINT(1) NOT NULL COMMENT '设备上下线状态，设备上线：1，设备下线：0',created TIMESTAMP DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',modified TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '修改时间');
# CREATE TABLE t_mobile_surveillance_events (event_serial_id INT PRIMARY KEY AUTO_INCREMENT COMMENT '埋点记录id',event_id VARCHAR(100) NOT NULL COMMENT '埋点名称，包括Live_watch_time(直播观看时长),icloud_watch_time(云回放观看时长云回放观看时长),Live_play_success(直播成功),Live_play_fail(直播失败),live_play_loading_time(直播加载时间)',event_duration BIGINT COMMENT '事件持续时长，单位：毫秒；根据event_id的不同含义，可能是用户观看时长、直播加载时长或为0',event_created TIMESTAMP NOT NULL COMMENT '事件创建时间',user_id VARCHAR(20) COMMENT '用户手机号',phone_type VARCHAR(50) COMMENT '手机类型',app_version VARCHAR(20) COMMENT '和家亲app版本号',network_type VARCHAR(10) COMMENT '网络，分为WIFI、4G等',mac_id CHAR(16) NOT NULL COMMENT '设备id',extra TEXT COMMENT '其他',created TIMESTAMP DEFAULT CURRENT_TIMESTAMP COMMENT '该条记录的创建时间',modified TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '记录修改时间');
#
### 以下是相关数据库中引用的一些数据信息:
#
# t_di_bind_device (id [1, 2, 3],user_id [135****1234, 136****2345, 137****3456],mac_id [4650********8886, 7067********4287, 5078********9631],mac_type [ZNIPC, CMCC-V7B, AP6PCM0],channel_id [3450*************80, 4562*************92, 4584*************16],bind [1, 1, 1],created [2023-12-13 00:00:03.0, 2023-12-13 00:00:04.0, 2023-12-13 00:00:13.0],modified [2023-12-13 00:00:03.0, 2023-12-13 00:00:04.0, 2023-12-13 00:00:13.0]);
# t_device_attributes (mac_id [1033582000000008, 1033582000000067, 1033582000000129],mac_type [YD4X1, YD4X1, YD4X1],supplier_id [101, 102, 101],supplier_name [中国移动, 华为, 中国移动],channel_name [浙江一级, 浙江一级, 电商渠道],channel_id [3450*************80, 4562*************92, 4700*************12],province_code [33, 11, -1],province_name [浙江, 北京, 全国]);
# t_user_packages (order_id [1014029, 2022050, 2022062],mac_id [0012*************4057, 0012*************3747, 0012*************5913],order_user [15955434323, 13879520069, 136****1235],province_code [0, 0, 0],city_code [0, 0, 0],created [2024-03-18 13:28:38.0, 2022-05-01 16:54:06.0, 2022-06-19 20:58:55.0],package_id [1fa7****960c, 1fa7****864f, 1fa7****960c],storage_type [1, 1, 1],state [1, 1, 1]);
# t_device_online_status (id [1, 2, 3],mac_id [0012*************4057, 0012*************3747, 0012*************5913],login [1, 1, 1],created [2024-07-01 00:00:00.0, 2024-07-01 00:00:00.0, 2024-07-01 00:00:00.0],modified [2024-07-01 03:24:30.0, 2024-07-01 23:55:40.0, 2024-07-01 23:20:40.0]);
# t_mobile_surveillance_events (event_serial_id [1, 2, 3, 4, 5],event_id [Live_watch_time, icloud_watch_time, Live_play_success, Live_play_fail, live_play_loading_time],event_duration [252, 1413, 0, 0, 2187],event_created [2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59],user_id [136****1235, 137****9579, 138****0407, 18832970987, 18260760987],phone_type [ANDROID, ANDROID, IOS, ANDROID, ANDROID],app_version [8.2.0, 8.4.0, 8.5.0, 8.2.0, 8.4.0],network_type [WiFi, 4G, WIFI, WiFi, WiFi],mac_id [0012*************4057, 0012*************3747, 0012*************5913, 0012*************5716, 0012*************4057],created [2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59],modified [2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59, 2024/6/30 23:59]);
#
### 问题:{question}
### SQL:
"""

def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def compute_combined_scores(distances, mask_similarities, threshold):
    return [(i, distances[i], mask_similarities[i]) for i in range(len(distances)) if mask_similarities[i] >= threshold]

def filter_prompt_lines(prompt, found_tables):
    filtered_prompt = []
    for line in prompt.split('\n'):
        if line.startswith("# ") and len(line) > 3:
            if any(table in line for table in found_tables):
                filtered_prompt.append(line)
        else:
            filtered_prompt.append(line)
    return "\n".join(filtered_prompt)

def main(your_question):
    your_question_embedding = encode_questions(embedding_model, [your_question])

    distances = compute_distances(your_question_embedding, question_embeddings)

    k = 5
    top_k_indices = distances.argsort()[:k]

    examples = "\n".join([f"### {questions[idx]}\n{queries[idx]}\n" for idx in top_k_indices]).strip()
    # print(f"生成pre-sql的所需示例:\n{examples}")

    schema = """
# t_di_bind_device (id, user_id, mac_id, mac_type, channel_id, bind, created, modified);
# t_device_attributes (mac_id, mac_type, supplier_id, supplier_name, channel_name, channel_id, province_code, province_name);
# t_user_packages (order_id, mac_id, order_user, province_code, city_code, created, package_id, storage_type, state);
# t_device_online_status (id, mac_id, login, created, modified);
# t_mobile_surveillance_events (event_serial_id, event_id, event_duration, event_created, user_id, phone_type, app_version, network_type, mac_id, extra, created, modified);
"""

    prompt = generate_prompt(examples, your_question)
    # print(f"pre-prompt为:\n{prompt}")

    response = get_completion(prompt)
    # print(f"pre-sql为:\n{response}")

    response_extract = extract_sql(response)
    
    table_names = {"t_di_bind_device", "t_device_attributes", "t_user_packages", "t_device_online_status", "t_mobile_surveillance_events"}
    found_tables = [table for table in table_names if table in response]

    columns = re.findall(r'\((.*?)\)', schema, re.DOTALL)
    columns = [col.strip().split(', ') for col in columns]
    column_set = {item for sublist in columns for item in sublist}

    response_mask = response_extract
    for table in table_names:
        response_mask = re.sub(r'\b' + table + r'\b', '<mask>', response_mask)

    for column in column_set:
        response_mask = re.sub(r'\b' + column + r'\b', '<unk>', response_mask)

    mask_similarities = [jaccard_similarity(response_mask, mask_query) for mask_query in mask_queries]

    threshold = 0.4
    combined_scores = compute_combined_scores(distances, mask_similarities, threshold)
    combined_scores = sorted(combined_scores, key=lambda x: x[1])
    top_pairs = [x for x in combined_scores if x[2] >= threshold]

    top_k_indices = [x[0] for x in top_pairs[:k]]

    examples_new = "\n".join([f"### {questions[idx]}\n{queries[idx]}\n" for idx in top_k_indices]).strip()
    # print(f"生成final-sql的所需示例:\n{examples_new}")

    prompt_new = generate_prompt(examples_new, your_question)
    prompt_final = filter_prompt_lines(prompt_new, found_tables)
    # print(f"final-prompt为:\n{prompt_final}")

    response_final = get_completion(prompt_final)
    return response_final

def recognize_speech():
    logging.info('识别中...')
    
    start_time = time.time()
    
    # 调用百度语音识别API进行语音识别
    ret = aip_speech.asr(audio_data, 'wav', 16000, {'dev_pid': 1536, })
    
    if ret and ret['err_no'] == 0:
        result = ''.join(ret['result'])
        print("识别结果:", result)
        end_time = time.time()
        print("识别时间:", end_time - start_time, "秒")
    else:
        print("识别失败:", ret['err_msg'])
    logging.info('end')

    return result if ret and ret['err_no'] == 0 else ""


def on_click(x, y, button, pressed):
    global recording, audio_data
    
    if pressed and not recording:
        logging.info('开始录音...')
        recording = True
        with mic as source:
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source, timeout=5, phrase_time_limit=10) # 按下开始录音
            audio_data = audio.get_wav_data()
    elif pressed and recording:
        logging.info('录音结束')
        recording = False
        return False

if __name__ == "__main__":
    logging.basicConfig(level=logging.DEBUG)

    # 创建语音识别对象
    r = sr.Recognizer()
    mic = sr.Microphone(sample_rate=16000)

    recording = False
    audio_data = None

    input_choice = input("请选择输入方式（1：音频输入，2：文字输入）：")

    if input_choice == '1':
        # 开始监听鼠标点击事件
        with mouse.Listener(on_click=on_click) as listener:
            listener.join()
        
        # 当录音完成后，进行识别
        if audio_data:
            your_question = recognize_speech()
        else:
            print("录音失败，请重试。")
            exit()
    elif input_choice == '2':
        your_question = input("请输入您的问题：")
    else:
        print("无效的选择，请选择1或2")
        exit()

    final_sql = main(your_question)
    print(f"final-sql为:\n{final_sql}")


INFO:root:开始录音...
INFO:root:录音结束
INFO:root:识别中...
DEBUG:urllib3.connectionpool:Resetting dropped connection: vop.baidu.com
DEBUG:urllib3.connectionpool:http://vop.baidu.com:80 "POST /server_api HTTP/11" 200 138
INFO:root:end


识别结果: 上半年的活跃用户数
识别时间: 0.5663425922393799 秒


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.28it/s]
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '4213f3298d82495a8fdaf5e838402493'}, 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "\n### 以下是基于类似问题提供的一些问题和相应的SQL查询的示例对：\n### 统计活跃客户的数量。\nSELECT count(*) FROM customer WHERE active = '1'\n\n### 有多少用户曾登录过？\nSELECT count(*) FROM users WHERE user_login  =  1\n\n### 有多少用户？\nSELECT count(*) FROM useracct\n\n### 有多少用户？\nSELECT count(*) FROM user_profiles\n\n### 每个用户有多少追随者？\nSELECT count(*) FROM follows\n### 仅通过SQLite SQL查询回答问题，不需要解释。您必须在确保正确性的同时最小化SQL执行时间。 \n### 给定以下数据库架构:\n#\n# CREATE TABLE t_di_bind_device (id INT PRIMARY KEY AUTO_INCREMENT COMMENT '唯一id',user_id VARCHAR(20) NOT NULL COMMENT '用户手机号，一个用户可绑定多个设备',mac_id CHAR(16) NOT NULL UNIQUE COMMENT '设备id，设备id长度一般为16位，每个设备都有唯一id',mac_type VARCHAR(50) COMMENT '设备型号',channel_id VARCHAR(50) COMMENT '设备渠道',bind TINYINT(1) NOT NULL COMM

final-sql为:
```sql
SELECT COUNT(DISTINCT user_id) 
FROM t_mobile_surveillance_events 
WHERE event_created BETWEEN '2024-01-01' AND '2024-06-30';
```
